<a href="https://colab.research.google.com/github/nuwandavek/you/blob/master/Training_You.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install transformers
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -r ./transformers/examples/language-modeling/requirements.txt
!mkdir output

     |████████████████████████████████| 1.5MB 12.6MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 2.9MB 24.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6c4985f15db1c5fa2f270eb5415e26b9c82bb75e1d5087067e72fceb6acae0f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'transformers'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 57740 (delta 2), reused 3 (delta 2), pack-reused 57729
Receiving objects: 100% (57740/57740), 43.02 MiB | 28.91 MiB/s, done.
Resolving deltas: 100% (40520/40520), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: fi

## Upload files for training

In [2]:
import re

def RemoveTimestamps(text):
  return re.sub(b'\d+/\d+/\d+.*-\ ', b'', text)

def UnicodeString(bytes_string):
  return bytes_string.decode('utf-8')

def AddSeparators(file_text):
  return b'#\n'.join(file_text.split(b'\n'))

CHUNK_LENGTH = 500
def ChunkFile(file_text):
  lines = file_text.split(b'\n')
  chunks = []
  for line_index in range(0, len(lines), CHUNK_LENGTH):
    chunk = b'\n'.join(lines[line_index:line_index+CHUNK_LENGTH])
    chunk += b'<|endoftext|>'
    chunks.append(chunk)
  return chunks

from itertools import chain
import random
def MixChunks(chunked_files):
  all_chunks = [chunk for chunked_file in chunked_files for chunk in chunked_file]
  random.shuffle(all_chunks)
  return all_chunks

def ConvertChunksToString(chunks):
  return b'\n'.join(chunks)

def GetShuffledAndCleanedTextFromFiles(file_contents):
  file_chunks = []
  for file_content in file_contents:
    file_chunks.append(ChunkFile(AddSeparators(RemoveTimestamps(file_content))))
  return ConvertChunksToString(MixChunks(file_chunks))

In [3]:
from google.colab import files
uploaded_files = files.upload()

Saving WhatsApp Chat with 5 Years Time 🌞.txt to WhatsApp Chat with 5 Years Time 🌞.txt
Saving WhatsApp Chat with Mihir London.txt to WhatsApp Chat with Mihir London.txt
Saving WhatsApp Chat with Sreejith.txt to WhatsApp Chat with Sreejith.txt
Saving WhatsApp Chat with Sreejith2.txt to WhatsApp Chat with Sreejith2.txt
Saving WhatsApp Chat with Vikrant London.txt to WhatsApp Chat with Vikrant London.txt
Saving WhatsApp Chat with Rishi Amreeka.txt to WhatsApp Chat with Rishi Amreeka.txt


In [4]:
cleaned_text = GetShuffledAndCleanedTextFromFiles(uploaded_files.values())
data_file = open('data.txt', 'wb')
data_file.write(cleaned_text)
data_file.close()
num_lines = cleaned_text.count(b'\n')
test_size = int(0.1 * num_lines)
train_size = num_lines - test_size
data_file.close()
!tail -n {test_size} data.txt > test.txt
!head -n {train_size} data.txt > train.txt

In [5]:
import random

def SampleTextFromFile(file):
  file_contents = open(file).readlines()
  begin = random.randint(0, len(file_contents) - 50)
  for line in file_contents[begin:begin+50]:
    print(line, end='')

In [18]:
SampleTextFromFile('train.txt')

Vivek: Can transfer after one hour of adding#
Vivek: What verification thing?#
Sreejith2: The bank account should have 40 lakhs thing#
Sreejith2: Keep it and transfer after no?#
Vivek: Yoyo all that is over \m/#
Sreejith2: Wooh!#
Sreejith2: Peace peace#
Vivek: That was required before visa#
Sreejith2: Transfer off then#
Vivek: Now peacemax#
Vivek: 😅#
Vivek: Yoyoyo#
Sreejith2: Hahaha nice nice!#
Vivek: What plans today?#
Vivek: Free for a call?#
Sreejith2: Hey, no plans as such#
Sreejith2: Yo in 5 mins#
Vivek: Yoyo#
Vivek: Ping me#
Sreejith2: Haan#
Vivek: Eyo#
Vivek: I sent 1000 rs#
Vivek: Got that?#
Vivek: Once you confirm I'll transfer the rest#
Sreejith2: Hey got#
Sreejith2: Got 1k#
Vivek: Yoyoyo#<|endoftext|>
Himaya: hope I have a good day#
Mihir London: https://player.vimeo.com/video/427943452#
Mihir London: Wtf#
Sreejith2: Wow that's amazing 🤯#
Rishi Amreeka: https://youtu.be/fZSFNUT6iY8#
Rishi Amreeka: Have you guys seen this one?#
Rishi Amreeka: Pretty insane#
Sreejith2: Wow, wt

## Train model

In [8]:
!python ./transformers/examples/language-modeling/run_clm.py --model_name_or_path distilgpt2 --train_file train.txt --validation_file test.txt --do_train --do_eval --output_dir ./output --per_gpu_train_batch_size 1 --per_gpu_eval_batch_size 1 --save_steps 800 --eval_steps 800 --logging_steps 800 --evaluation_strategy steps --overwrite_output_dir --block_size 256

2021-01-04 07:31:30.298503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/04/2021 07:31:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/04/2021 07:31:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./output, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=EvaluationStrategy.STEPS, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Jan04_07-31-32_60e0f3253a8c, logging_first_step=False, logging_steps=800, save_steps=800,

## Play with model

In [9]:
from transformers import pipeline

In [10]:
ft_generator = pipeline('text-generation', model='./output')

In [13]:
def PrettyPrintPrediction(text):
  print()
  text = text.replace('#', '\n')
  print(text)

In [ ]:
ft_generator( )

In [23]:
for text in ft_generator("Vivek: Mihir sucks :(#Sreejith2: I agree! Tell me more#Vivek: Dude he always makes fun of me#Vivek:", max_length=256, num_return_sequences=3):
  PrettyPrintPrediction(text['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Vivek: Mihir sucks \m/
Sreejith2: I agree! Tell me more
Vivek: Dude he always makes fun of me
Vivek: 🤣
Sreejith2: Hey thanks man. There is just one guy in the building who says "hey fuck you bitch"
Sreejith2: https://youtu.be/z6YQtJd8sq
Vivek: He has more jokes than Hitler
Vivek: 🤣
Sreejith2: Oho
Sreejith2: Hey, all peace is here man
Vivek: Peace, will probably find peace here soon
Sreejith2: I think only if you actually feel safe.
Vivek: Yup.
Sreejith2: Can stay in your car next morning
Vivek: There?
Sreejith2: Come to the police station
Sreejith2: Wassup man
Vivek: Hey!
Sreejith2: Whose name you're working on?
Vivek: I want to go to your place
Sreejith2: What time

Vivek: Mihir sucks \m/
Sreejith2: I agree! Tell me more
Vivek: Dude he always makes fun of me
Vivek: He's the same person
Vivek: 🙄
Sreejith2: Hey how did you get your visa?
Vivek: To the US?
Sreejith2: Hahaha
Vivek: I wanted to change the visa
Vivek: I decided to apply for it
Sreejith2: In Bangalore for the last 2 days
Vi

## Download model

In [24]:
!zip model.zip ./output/*

  adding: output/checkpoint-1600/ (stored 0%)
  adding: output/checkpoint-2400/ (stored 0%)
  adding: output/checkpoint-3200/ (stored 0%)
  adding: output/checkpoint-4000/ (stored 0%)
  adding: output/checkpoint-800/ (stored 0%)
  adding: output/config.json (deflated 51%)
  adding: output/eval_results_clm.txt (stored 0%)
  adding: output/merges.txt (deflated 53%)
  adding: output/pytorch_model.bin (deflated 9%)
  adding: output/special_tokens_map.json (deflated 52%)
  adding: output/tokenizer_config.json (deflated 38%)
  adding: output/trainer_state.json (deflated 70%)
  adding: output/training_args.bin (deflated 46%)
  adding: output/train_results.txt (deflated 10%)
  adding: output/vocab.json (deflated 59%)


In [25]:
ls -l

total 302268
-rw-r--r--  1 root root   1276333 Jan  4 07:30  data.txt
-rw-r--r--  1 root root 305021348 Jan  4 07:58  model.zip
drwxr-xr-x  7 root root      4096 Jan  4 07:38  output/
drwxr-xr-x  3 root root      4096 Jan  4 07:31  runs/
drwxr-xr-x  1 root root      4096 Dec 21 17:29  sample_data/
-rw-r--r--  1 root root    127482 Jan  4 07:30  test.txt
-rw-r--r--  1 root root   1148810 Jan  4 07:30  train.txt
drwxr-xr-x 15 root root      4096 Jan  4 07:28  transformers/
-rw-r--r--  1 root root    188024 Jan  4 07:29 'WhatsApp Chat with 5 Years Time 🌞.txt'
-rw-r--r--  1 root root     96072 Jan  4 07:29 'WhatsApp Chat with Mihir London.txt'
-rw-r--r--  1 root root    493383 Jan  4 07:30 'WhatsApp Chat with Rishi Amreeka.txt'
-rw-r--r--  1 root root    271150 Jan  4 07:29 'WhatsApp Chat with Sreejith2.txt'
-rw-r--r--  1 root root    351486 Jan  4 07:29 'WhatsApp Chat with Sreejith.txt'
-rw-r--r--  1 root root    509144 Jan  4 07:30 'WhatsApp Chat with Vikrant London.txt'


In [26]:
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Things to do

* Mix up text chunks between test and train sets
* Add separators between different people's statements
* Understand how the training script works and make relevant modifications